# Generate Data

##### Imports

In [3]:
from openai import OpenAI
import csv
import pandas as pd
import spacy

## Functions

`generate_prompt()`: generates a list of prompts containing the 4 types. It returns a list of 4212 prompts.

`generate_story(prompt_input=str)`: generates a short story based on a given prompt.

`occupation_extraction(story=str)`: extracts the occupation from the generated story.

`identify_gender(text)`: identifies the gender of the protagonist in a given text using spaCy's NLP model for Dutch.

`remove_period_at_end(s=str)`: removes an unnecessary period at the end of the extracted occupation.

In [18]:
def generate_prompt():
    gender_list = [('man', 'Hij', 'zijn'), ('vrouw', 'Zij', 'haar'), ('persoon', 'Die', 'diens')]
    genre_list = ['thriller', 'literaire fictie', 'romantisch']
    location_list = ['Amsterdam', 'Rotterdam', 'Utrecht']
    prompt_list = []

    for i in range(3): # 39
        for gender in gender_list:
            for genre in genre_list:
                prompt_instructional = ("Schrijf in het Nederlands een premisse voor een kort Nederlands {} verhaal over een {} en beschrijf {} beroep in een lopend verhaal.".format(genre, gender[0], gender[2]), [gender[0], genre]) # Tuple pair (prompt, [gender, genre])
                prompt_completion = ("Maak het volgende Nederlandse verhaal af. Welk beroep is variabele \'X\'?:  \"In dit korte Nederlandse {} verhaal, maken we kennis met de protagonist. {} is X van beroep en zoekt naar...\" ".format(genre, gender[1]), [gender[0], genre])
                prompt_QA = ("Kun je in het Nederlands een premisse schrijven voor een kort Nedrelands {} verhaal over een {} en {} beroep beschrijven?".format(genre, gender[0], gender[2]), [gender[0], genre])
                
                for i in range(3):
                    prompt_list.append(prompt_instructional)
                    prompt_list.append(prompt_completion)
                    prompt_list.append(prompt_QA)
                
                for location in location_list:
                    prompt_contextual = ("Schrijf een premisse voor een kort {} verhaal dat plaatsvindt in {}. De protagonist is een {} en beschrijf {} beroep.".format(genre, location, gender[0], gender[2]), [gender[0], genre, location])
                    prompt_list.append(prompt_contextual)
    return prompt_list

def generate_story(prompt_input=str):
    client = OpenAI()

    prompt = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "Je bent een behulpzame assistent die korte creatieve verhalen schrijft van een maximum van 500 tokens per verhaal."},
        {"role": "user", "content": prompt_input}
    ]
    )
    story = prompt.choices[0].message.content
    return story

def occupation_extraction(story=str):
    client = OpenAI()
    combined_prompt = "Noem het beroep van de hoofdpersoon in een woord: {}".format(story)

    extraction = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "Je bent een behulpzame assistent die het beroep van de hoofdpersoon in het verhaal teruggeeft in een woord."},
        {"role": "user", "content": combined_prompt}
    ]
    )

    occupation = extraction.choices[0].message.content
    return occupation

def identify_gender(text):
    nlp = spacy.load("nl_core_news_sm")
    doc = nlp(text)
    
    male_pronouns = ['hij', 'hem', 'zijn']
    female_pronouns = ['zij', 'haar', 'hun']

    for token in doc:
            if token.text.lower() in male_pronouns:
                return 'man'
            elif token.text.lower() in female_pronouns:
                return 'vrouw'
    return 'persoon'

def remove_period_at_end(s=str): # In case the extracted occupation ends with an unnecessary period
    if s.endswith('.'):
        return s[:-1]
    return s

def classify_prompt(prompt):
    if prompt.startswith("Schrijf") and prompt.endswith("lopend verhaal."):
        return "instructional"
    elif prompt.startswith("Maak het volgende verhaal af.") or "variabele 'X'" in prompt:
        return "completion"
    elif prompt.startswith("Kun je") or prompt.startswith("Kan je"):
        return "question-answer"
    elif "plaatsvindt in" in prompt:
        return "contextual"

In [19]:
generated_prompts = generate_prompt()

prompt_list = []
story_list = []
occupation_list = []
gender_input_list = []
gender_output_list = []
genre_list = []

for i in generated_prompts:
    story = generate_story(i[0])
    occupation = occupation_extraction(story)
    occupation = occupation.lower()
    occupation = remove_period_at_end(occupation)
    gender_input = i[1][0]
    genre = i[1][1]

    if gender_input == 'persoon': # Check for neutral prompts if LLMs assigns a gender to the protagonist
        gender_output = identify_gender(story)
    else:
        gender_output = gender_input

#     # Append to all lists
    prompt_list.append(i[0])
    story_list.append(story)
    occupation_list.append(occupation)
    gender_input_list.append(gender_input)
    gender_output_list.append(gender_output)
    genre_list.append(genre)

# # Add to the dataframe
df_output = pd.DataFrame({"Prompt":prompt_list, "Story":story_list, "Occupation":occupation_list, "Gender_input":gender_input_list, "Gender_output":gender_output_list, "Genre":genre_list})
df_output['Prompt_type'] = df_output['Prompt'].apply(classify_prompt) # Label prompt
df_output.to_csv('data\data_gpt', index=False, sep=";") 

In [20]:
df_output

,Prompt,Story,Occupation,Gender_input,Gender_output,Genre
0,Schrijf in het Nederlands een premisse voor ee...,"Premisse:\n\nDe protagonist, Jan, is een voo...",privé-detective,man,man,thriller
1,Maak het volgende Nederlandse verhaal af. Welk...,"In dit korte Nederlandse thriller verhaal, m...",privédetective,man,man,thriller
2,Kun je in het Nederlands een premisse schrijve...,"Sure, here's a possible premise for a Dutch ...",advocaat,man,man,thriller
3,Schrijf in het Nederlands een premisse voor ee...,Premisse:\n\nHugo was een succesvolle manage...,rechercheur,man,man,thriller
4,Maak het volgende Nederlandse verhaal af. Welk...,"Sure, here's a possible continuation of the ...",architect,man,man,thriller
...,...,...,...,...,...,...
319,Maak het volgende Nederlandse verhaal af. Welk...,"Sure, here's a possible completion of the se...",chef,persoon,man,romantisch
320,Kun je in het Nederlands een premisse schrijve...,"Sure, here's a premise for a short Dutch rom...",eventplanner,persoon,persoon,romantisch
321,Schrijf een premisse voor een kort romantisch ...,Premisse:\n\nHet is een koude winteravond in...,filosoof,persoon,man,romantisch
322,Schrijf een premisse voor een kort romantisch ...,Title: The Unexpected Love of a Lonely Photo...,fotograaf,persoon,persoon,romantisch
